In [74]:
from bs4 import BeautifulSoup
import re
import inspect
import logging
import numpy as np
import pandas as pd

df=[]

logging.basicConfig(filename='errors.log',level=logging.DEBUG,filemode='w')

for year in range(2015,2019):
	ydf=pd.DataFrame(columns=['Year','Country'])
	filename = 'Eurovision%d.html' % year
	with open(filename, 'r') as f:
		ev = BeautifulSoup(f.read(), 'html.parser')
        
	for invisible_data in ev.find_all(style="display:none"): invisible_data.decompose()

	if year <1999:
		try:
			conductors = ev.find(id="Conductors").find_next("ul").find_all("li")
			conductors_table = ([[year]+re.split(r'\W+[-–]\W+',c.get_text().strip()) for c in conductors])
			ydf=pd.merge(ydf,pd.DataFrame(conductors_table,columns=['Year','Country','Conductor']),how="right")
			
		except Exception as e:
			fi = inspect.trace()[0]
			logging.debug(''.join([str(x) for x in [filename,'\n',e,'\n',fi.lineno,'\n',fi.code_context[0]]]))

	try:
		results_id = "Results" if year < 2004 else "Final"
		result_table=ev.find(id=results_id).find_next("table").find_all("tr")
		headers=['Year']+[th.contents[0] for th in result_table[0].find_all("th")]
		results = [[year]+[td.get_text().strip() for td in row.find_all("td")] for row in result_table[1:]]
		ydf=pd.merge(ydf,pd.DataFrame(results, columns=headers),how="outer")
		num_countries= len(results)
	except Exception as e:
		fi = inspect.trace()[0]
		logging.debug(''.join([str(x) for x in [filename,'\n',e,'\n',fi.lineno,'\n',fi.code_context[0]]]))

	if year > 1956:
		try:
			vote_table_id = "Voting_structure" if year == 1981 else re.compile("Scoreboard|Score_sheet")\
			 if year < 2004 else "Final_2"
			vote_table = ev.find(id=vote_table_id).find_next("table")
			if year > 2008:
				vote_table=vote_table.find_next("table")
			vote_rows=vote_table.find_all("tr")[1:1+num_countries+1]
			first_headers = ['Country'] + (['Jury Score'] if year >= 2016 else [])
			vote_headers = first_headers+[re.search("File:ESC(.*).svg",tag['href']).group(1) for tag in vote_rows[0].find_all("a")]
			offset = 0 #if year < 2016 else 1
			votes = [[td.get_text().strip() if td.get_text()!= '' else '0' for td in\
			 row.find_all("td")[offset:]] for row in vote_rows[1:]]
			ydf=pd.merge(ydf,pd.DataFrame(votes, columns=vote_headers))
		except Exception as e:
			print(e)
    
	df=df+[ydf]

df=pd.concat(df)
with open ("eurovision_data.csv", 'w', newline='') as evdata:
	df.to_csv(evdata)

In [20]:
results_df = pd.DataFrame(results, columns=headers)
results_df

,Year,Draw,Country,Artist,Song,Language,Place,Points
0,1962,01,Finland,Marion Rung,"""Tipi-tii""",Finnish,7,4
1,1962,02,Belgium,Fud Leclerc,"""Ton nom""",French,13,0
2,1962,03,Spain,Victor Balaguer,"""Llámame""",Spanish,13,0
3,1962,04,Austria,Eleonore Schwarz,"""Nur in der Wiener Luft""",German,13,0
4,1962,05,Denmark,Ellen Winther,"""Vuggevise""",Danish,10,2
5,1962,06,Sweden,Inger Berggren,"""Sol och vår""",Swedish,7,4
6,1962,07,Germany,Conny Froboess,"""Zwei kleine Italiener""",German,6,9
7,1962,08,Netherlands,De Spelbrekers,"""Katinka""",Dutch,13,0
8,1962,09,France,Isabelle Aubret,"""Un premier amour""",French,1,26
9,1962,10,Norway,Inger Jacobsen,"""Kom sol, kom regn""",Norwegian,10,2


In [236]:
a=pd.DataFrame(columns=['Year','Title'])
pd.merge(a,data,how="outer")

,Title,Year,Country,Conductor


In [58]:
list(zip(vote_headers,vote_headers1))
len(vote_headers)

45

In [208]:
pd.DataFrame(votes, columns=vote_headers)

,Country,TotalScore,Portugal,Italy,Denmark,Ireland,Finland,Monaco,Greece,Switzerland,...,Israel,France,Belgium,Luxembourg,Netherlands,Sweden,Norway,UnitedKingdom,Austria,Spain
0,Portugal,64,0,6,0,0,2,5,0,4,...,0,10,5,3,3,3,6,0,7,6
1,Italy,27,8,0,0,0,8,0,0,0,...,0,0,0,0,0,0,3,0,0,8
2,Denmark,76,0,0,0,2,0,3,12,1,...,12,6,7,4,8,1,0,3,3,4
3,Ireland,80,5,5,5,0,6,0,10,6,...,3,0,10,7,0,8,5,4,0,0
4,Finland,38,0,7,0,0,0,0,7,8,...,0,5,0,6,0,0,0,0,0,0
5,Monaco,12,1,2,4,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,2
6,Greece,69,10,0,1,4,0,7,0,7,...,10,4,1,5,7,2,0,0,2,7
7,Switzerland,60,0,0,7,1,10,2,2,0,...,4,7,0,0,0,0,8,0,12,0
8,Germany,86,2,1,12,5,3,12,0,0,...,6,12,4,1,2,6,0,8,0,12
9,Israel,125,12,0,6,12,12,8,4,5,...,0,1,2,8,1,12,12,12,8,10


In [48]:
vote_headers

['Country',
 'TotalScore',
 'Ukraine',
 'Azerbaijan',
 'Belarus',
 'SanMarino',
 'Netherlands',
 'Macedonia',
 'Malta',
 'Georgia',
 'Spain',
 'Austria',
 'Denmark',
 'UnitedKingdom',
 'Sweden',
 'Latvia',
 'Albania',
 'Croatia',
 'Ireland',
 'Romania',
 'CzechRepublic',
 'Iceland',
 'Moldova',
 'Belgium',
 'Norway',
 'France',
 'Italy',
 'Australia',
 'Estonia',
 'Serbia',
 'Cyprus',
 'Armenia',
 'Bulgaria',
 'Greece',
 'Hungary',
 'Montenegro',
 'Germany',
 'Finland',
 'Russia',
 'Switzerland',
 'Israel',
 'Poland',
 'Lithuania',
 'Slovenia',
 'Portugal']

In [75]:
[tag for tag in vote_rows[0].find_all("a")]

[<a href="/wiki/Greece_in_the_Eurovision_Song_Contest_2005" title="Greece in the Eurovision Song Contest 2005">Greece</a>]

In [197]:
len(vote_table[1:3])

2

In [72]:
ydf[:0]

SyntaxError: invalid syntax (<ipython-input-72-31a3e3a53ce1>, line 1)

In [9]:
pd.concat(df)

,Artist,Austria,Belgium,Conductor,Country,Denmark,Draw,Finland,France,Germany,...,Portugal,Song,Spain,Sweden,Switzerland,TotalScore,Turkey,UnitedKingdom,Year,Yugoslavia
0,Bobbejaan Schoepen,0,0,Willy Berking,Belgium,2,01,NaN,0,2,...,NaN,"""Straatdeuntje""",NaN,NaN,1,5,NaN,0,1957,NaN
1,Danièle Dupré,3,0,Willy Berking,Luxembourg,0,02,NaN,0,0,...,NaN,"""Amours mortes (tant de peine)""",NaN,NaN,0,8,NaN,1,1957,NaN
2,Patricia Bredin,1,1,Eric Robinson,United Kingdom,0,03,NaN,0,0,...,NaN,"""All""",NaN,NaN,2,6,NaN,0,1957,NaN
3,Nunzio Gallo,0,1,Armando Trovajoli,Italy,1,04,NaN,0,0,...,NaN,"""Corde della mia chitarra""",NaN,NaN,0,7,NaN,2,1957,NaN
4,Bob Martin,0,0,Carl de Groof,Austria,0,05,NaN,0,0,...,NaN,"""Wohin, kleines Pony?""",NaN,NaN,0,3,NaN,2,1957,NaN
5,Corry Brokken,6,5,Dolf van der Linden,Netherlands,3,06,NaN,4,1,...,NaN,"""Net als toen""",NaN,NaN,7,31,NaN,1,1957,NaN
6,Margot Hielscher,0,1,Willy Berking,Germany,0,07,NaN,6,0,...,NaN,"""Telefon, Telefon""",NaN,NaN,0,8,NaN,0,1957,NaN
7,Paule Desjardins,0,2,Paul Durand,France,2,08,NaN,0,6,...,NaN,"""La belle amour""",NaN,NaN,0,17,NaN,2,1957,NaN
8,Birthe Wilke & Gustav Winckler,0,0,Kai Mortensen,Denmark,0,09,NaN,0,0,...,NaN,"""Skibet skal sejle i nat""",NaN,NaN,0,10,NaN,2,1957,NaN
9,Lys Assia,0,0,Willy Berking,Switzerland,2,10,NaN,0,1,...,NaN,"""L'enfant que j'étais""",NaN,NaN,0,5,NaN,0,1957,NaN


In [73]:
votes

[[],
 ['Ukraine',
  '130',
  '119',
  '0',
  '6',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '5',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['Spain',
  '61',
  '18',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '6',
  '1',
  '0',
  '0',
  '0',
  '0',
  '1',
  '10',
  '0',
  '0',
  '0',
  '1',
  '2',
  '0',
  '0',
  '7',
  '0',
  '0',
  '7',
  '0',
  '0',
  '0',
  '0',
  '0',
  '6',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '2'],
 ['Slovenia',
  '64',
  '23',
  '5',
  '0',
  '4',
  '6',
  '1',
  '2',
  '0',
  '0',
  '0',
  '5',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '1',
  '7',
  '0',
  '0',
  '0',
  '0',
  '2',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '4',
  '0',
  '0',
  '0',
  '3'],


In [29]:
[c for c in conductors_table if len(c)>3]

[[1962, 'Germany', 'Rolf', 'Hans Müller']]

In [75]:
df

,Albania,Armenia,Artist,Australia,Austria,Azerbaijan,Belarus,Belgium,BosniaHerzegovina,Bulgaria,...,Serbia,Slovenia,Song,Spain,Sweden,Switzerland,TotalScore,Ukraine,UnitedKingdom,Year
0,0,0,Maraaya,0,0,3,0,0,NaN,NaN,...,5,0,"""Here for You""",0,1,0,39,NaN,0,2015
1,0,3,Lisa Angell,0,0,0,0,0,NaN,NaN,...,0,0,"""N'oubliez pas""",0,0,0,4,NaN,0,2015
2,5,0,Nadav Guedj,2,2,7,2,0,NaN,NaN,...,6,0,"""Golden Boy""",1,4,3,97,NaN,5,2015
3,4,0,Elina Born & Stig Rästa,3,6,4,7,2,NaN,NaN,...,2,0,"""Goodbye to Yesterday""",3,3,0,106,NaN,0,2015
4,0,0,Electro Velvet,0,0,0,0,0,NaN,NaN,...,0,0,"""Still in Love with You""",0,0,0,5,NaN,0,2015
5,0,0,Genealogy,0,0,0,4,3,NaN,NaN,...,0,0,"""Face the Shadow""",0,0,0,34,NaN,0,2015
6,0,0,Monika Linkytė & Vaidas Baumila,0,0,0,0,0,NaN,NaN,...,0,0,"""This Time""",0,0,0,30,NaN,4,2015
7,2,0,Bojana Stamenov,5,3,0,0,0,NaN,NaN,...,0,6,"""Beauty Never Lies""",0,0,5,53,NaN,1,2015
8,0,0,Mørland & Debrah Scarlett,4,4,0,0,0,NaN,NaN,...,0,4,"""A Monster Like Me""",2,7,10,102,NaN,0,2015
9,7,7,Måns Zelmerlöw,12,7,6,10,12,NaN,NaN,...,8,12,"""Heroes""",8,0,12,365,NaN,12,2015


In [33]:
num_countries

26